## Library Imports

In [1]:
import numpy as np
# pip install galois
import galois
from numpy.random import default_rng
import hashlib

In [2]:
GF256 = galois.GF(2**8)   # Field is taken as Galois field 256 with irreducible polynomial as x^8 + x^4 + x^3 + x^2 + 1

In [3]:
GF256.properties

'Galois Field:\n  name: GF(2^8)\n  characteristic: 2\n  degree: 8\n  order: 256\n  irreducible_poly: x^8 + x^4 + x^3 + x^2 + 1\n  is_primitive_poly: True\n  primitive_element: x'

## generate_gf256_message function generates the hash of a message of any length in GF256 field (for experiment, the length is kept between 1 and 100 bytes)

In [4]:
def generate_gf256_message(o):
    # Step 1: Generate a random message of random length in hexadecimal form
    random_message = np.random.bytes(np.random.randint(1, 100))  # message length between 1 and 100 bytes

    # Step 2: Hash the message using SHAKE256 or any suitable hash function
    hash_output = hashlib.shake_256(random_message).digest(o)  # output size `o` bytes (number of oil variables)

    # Step 3: Convert the hash output (bytes) into a NumPy array (in the space range 0-255)
    hash_array = np.frombuffer(hash_output, dtype=np.uint8)

    # Step 4: Convert hash_array to a GF(256) field element and reshape it to a column vector
    m = GF256(hash_array).reshape(o, 1)

    return random_message, m

In [5]:
generate_gf256_message(5)

(b'\xec,\xd9d\xf6\x99\xccS(\x06\xe7\xf6\xe2\x19^v\xc8\xb4\x16"\xcd\x1f\xb8\x06B\x1e[1\x00d_FI\xdb^pW\xb6"\xbfrV\xfa#\xb6\xc9\xeb\xb73\xde=',
 GF([[230],
     [112],
     [ 48],
     [102],
     [ 63]], order=2^8))

In [6]:
def generate_random_polynomial(o,v):   # Generate a random polynomial(quadratic) in o+v variables that vanishes in oil space
    f_i = np.vstack(
    (np.hstack((np.zeros((o,o),dtype=np.uint8),np.random.randint(256, size=(o,v), dtype=np.uint8))),
    np.random.randint(256, size=(v,v+o),dtype=np.uint8))
    )
    f_i_triu = np.triu(f_i)
    return GF256(f_i_triu)

In [7]:
generate_random_polynomial(4,6)  # The vinegar vinegar entries (bottom end 6*6 matrix) is upper triangular as v_i v_j variables are commutative.

GF([[  0,   0,   0,   0, 106, 134,  90,  18,  27,   7],
    [  0,   0,   0,   0, 179, 118, 223,  78, 113, 109],
    [  0,   0,   0,   0,  10,  90, 203, 144, 198,  23],
    [  0,   0,   0,   0, 217,  62, 123, 231,  97, 238],
    [  0,   0,   0,   0, 241, 195, 230,  76, 177, 106],
    [  0,   0,   0,   0,   0, 230, 188, 152, 232,   2],
    [  0,   0,   0,   0,   0,   0, 152,  62, 181,  76],
    [  0,   0,   0,   0,   0,   0,   0, 218, 175, 152],
    [  0,   0,   0,   0,   0,   0,   0,   0,  48,  82],
    [  0,   0,   0,   0,   0,   0,   0,   0,   0, 126]], order=2^8)

In [8]:
def generate_central_map(o,v):  #Generates the map F, which consists of o random polynomials (using generate_random_polynomial).
    F = []
    for _ in range(o):
        F.append(generate_random_polynomial(o,v))
    return F

In [9]:
F = generate_central_map(4,6)
F

[GF([[  0,   0,   0,   0, 101, 181, 248, 127,  41,  51],
     [  0,   0,   0,   0, 206, 121, 202, 197, 236,  87],
     [  0,   0,   0,   0, 133, 208, 150,  31, 145, 148],
     [  0,   0,   0,   0,  61,  46,  48, 141, 123, 177],
     [  0,   0,   0,   0, 249, 106, 171, 201, 174,  12],
     [  0,   0,   0,   0,   0,  75, 183,  72, 147, 249],
     [  0,   0,   0,   0,   0,   0, 252,  72,  15, 253],
     [  0,   0,   0,   0,   0,   0,   0, 110, 254, 250],
     [  0,   0,   0,   0,   0,   0,   0,   0, 127, 198],
     [  0,   0,   0,   0,   0,   0,   0,   0,   0, 207]], order=2^8),
 GF([[  0,   0,   0,   0, 240, 131, 225, 117, 201, 154],
     [  0,   0,   0,   0,  62,  42, 150,  22, 131, 101],
     [  0,   0,   0,   0,  52, 195, 107, 138, 231,  68],
     [  0,   0,   0,   0,  49, 142, 134, 111,  62, 128],
     [  0,   0,   0,   0, 177,  66, 106,  39, 144, 251],
     [  0,   0,   0,   0,   0, 251, 190,  12,  15,  34],
     [  0,   0,   0,   0,   0,   0, 238, 143, 178, 210],
     [  0,   0,   

## generate_central_map(o,v) creates 'o' number of quadratic polynomials in o+v variables that vanishes in oil space. Thus, this central map creates a o-tuple output, where each entry is a quadratic polynomial in o+v variable. 

In [10]:
def generate_affine_L(o,v):   #Generates an affine map in o+v variables
    found = False
    while not found:
        try:
            L_n = np.random.randint(256, size=(o+v,o+v), dtype=np.uint8)
            L = GF256(L_n)
            L_inv = np.linalg.inv(L)
            found = True
        except:
            found = False
    return L, L_inv    # Outputs the affine map and its inverse as well.

In [11]:
generate_affine_L(4,6)

(GF([[ 23,  23, 243, 120, 245, 122, 241,  94, 245, 218],
     [ 63,  22, 185,  27, 101, 191, 215,  49, 140, 153],
     [186, 198, 224,  46,  53,  38,  31, 187, 183,  33],
     [113,  44, 237, 169,  11, 160, 221,  53,  54, 176],
     [ 54, 209, 246, 127, 138,  42, 221, 159, 250, 220],
     [ 97, 117, 122, 239, 192, 105,  57,  57,  33,  31],
     [206, 196, 116,  93, 210,  42, 222,   7, 138,  21],
     [168,  89, 161,  55,  82,  50,  38, 222, 239, 125],
     [ 65,  72, 136, 220, 121,  73, 103, 206,  23,  24],
     [126,  31, 137,  22,  73, 173,  24, 254, 117, 222]], order=2^8),
 GF([[ 93, 253,  40,  92, 246, 102, 180, 193,  42, 204],
     [253, 117, 204,  86,   9, 160,  49, 117,   4, 214],
     [ 10,   4, 105, 216, 174,  82, 167, 218, 242, 148],
     [199, 105, 235, 199,  69, 115, 244,  77,  86, 250],
     [162, 152,  27, 154,  41, 102,  48, 131, 134, 135],
     [198,  12,  98,  93,  87, 199, 212,  43,  59,  58],
     [ 83, 155, 165,   7,   4,  98, 104, 105,  63, 192],
     [ 95, 238,  4

In [12]:
def generate_random_vinegar(v):     # Randomly generated vinegar variable
    vv = np.random.randint(256, size=v, dtype=np.uint8)
    rvv = GF256(vv)
    return rvv

def generate_random_vector(v):      # Randomly generated vector
    rv = np.random.randint(256, size=v, dtype=np.uint8)
    rand_v = GF256(rv)
    return rand_v

def generate_random_permutation_matrix(v):
    rng = default_rng()
    # Generate a random permutation matrix
    perm = np.eye(v, dtype=np.uint8)[rng.permutation(v)]
    
    # Replace 1s with random non-zero values from 1 to 255 (since 0 is not allowed)
    random_values = rng.integers(1, 256, size=(v, v), dtype=np.uint8)
    rand_Q = perm * random_values
    
    # Convert to GF256 (assuming GF256 is your finite field implementation)
    rand_Q = GF256(rand_Q)
    
    return rand_Q

In [13]:
rvv = generate_random_vinegar(6)
rvv

GF([135,  75,  24, 102,  45, 133], order=2^8)

In [14]:
generate_random_vector(4)

GF([158, 223,  50, 246], order=2^8)

In [15]:
generate_random_permutation_matrix(4)

GF([[  0,   0,  39,   0],
    [  0, 166,   0,   0],
    [ 34,   0,   0,   0],
    [  0,   0,   0,  47]], order=2^8)

# sub_vinegar_aux(rvv,f,o,v) enters the vinegar values to the polynomials and make it a linear map in oil space. 

In [16]:
def sub_vinegar_aux(rvv,f,o,v):
    coeffs = GF256([0]* (o+1))   # Creates a zero row of (o+1) size to store the coefficients. 'o' coefficients for 'o' oil variables. 1 for constant.
    # oil variables are in 0 <= i < o
    # vinegar variables are in o <= i < n 
    for i in range(o+v):
        for j in range(i,o+v):
            # by cases
            # oil and oil do not mix
            if i < o and j < o:
                pass
            # vinegar and vinegar contribute to a constant
            elif i >=o and j >= o:
                ij = GF256(f[i,j])
                vvi = GF256(rvv[i-o])
                vvj = GF256(rvv[j-o])
                coeffs[-1] += np.multiply(np.multiply(ij,vvi), vvj)
            # have mixed oil and vinegar variables that contribute to o_i coeff
            elif i < o and j >= o:
                ij = GF256(f[i,j])
                vvj = GF256(rvv[j-o])
                coeffs[i] += np.multiply(ij,vvj)
            # condition is not hit as we have covered all combos
            else:
                pass
    return coeffs


def sub_vinegar(rvv,F,o,v):   # The final map after substituing the vinegar variables
    subbed_rvv_F = []
    for f in F:
        subbed_rvv_F.append(sub_vinegar_aux(rvv,f,o,v))
    los = GF256(subbed_rvv_F)
    return los

In [17]:
sub_vinegar(rvv,F,4,6)

GF([[ 31, 150,  52, 124,  38],
    [162, 243,  82, 152, 218],
    [156,  96, 188, 240, 121],
    [ 74,  35,  52, 180,  29]], order=2^8)

In [18]:
# main functions

def generate_private_key(o,v): 
    F = generate_central_map(o,v)
    L, L_inv = generate_affine_L(o,v)
    return F, L, L_inv

def generate_public_key(F,L):
    L_T = np.transpose(L)
    P = []
    for f in F:
        s1 = np.matmul(L_T,f)
        s2 = np.matmul(s1,L)
        P.append(s2) 
    return P

In [19]:
F, L, L_inv = generate_private_key(4,6)
generate_public_key(F,L)

[GF([[ 61, 131,  73, 210, 247, 134, 187,  61,  97, 204],
     [161,  48,  14,  55, 251, 100,   9, 115, 174, 130],
     [ 80,  72, 223, 141,  27,   0, 136, 238,  10,  79],
     [146, 115, 116, 217,  38, 139,  42,  34,  77, 162],
     [181, 146,  15,  27, 184, 239,  40,  39,  64, 203],
     [253,  20, 188,  77, 215,  59, 145, 127, 231, 185],
     [145, 204, 222, 213,   0, 164, 253, 142,  89, 177],
     [172, 225, 154, 118, 196,  27,  79,  50,  17, 184],
     [162,   6,  89,  55,  58, 147, 236,  42, 173,  22],
     [199,  73, 248,  37,  61, 172,  62, 246,  75,  85]], order=2^8),
 GF([[ 90, 141, 150, 172, 223, 192, 216, 115, 192,  97],
     [ 65, 101, 183, 227, 195, 166, 149, 120,  92, 100],
     [244, 157, 253, 254, 139, 150, 193, 136,  29, 231],
     [199, 200, 242, 252, 236,  32,  96,  87, 182, 166],
     [173, 106, 183,   3,  22, 154, 203, 200, 164,  57],
     [ 53, 108, 127, 248, 231,  17,  88,  33, 146, 166],
     [202, 152, 135,  13, 185,  66, 122, 114, 226, 244],
     [  2,  49,  1

## We compute our public verification key by computing L ∘ f for each f in F. This is made easy as we can do this in quadratic form, thus each f_p in P is computed as L_T · f · L for each f in F where L_T is the transpose of L.

In [20]:
def sign_outsource(M,y,o):   # Creates the requirements for outsourcing scenarios
    rand_v = generate_random_vector(o).reshape(o,1)
    rand_Q = generate_random_permutation_matrix(o)
    Enc_M = np.matmul(rand_Q,M)
    Enc_y = np.matmul(rand_Q,np.add(y,np.matmul(M,rand_v)))
    return rand_v, Enc_M , Enc_y

In [21]:
def retrace_sign(rvv,Enc_sol,rand_v):  # Generates the signature.
    sol = np.subtract(Enc_sol,rand_v)
    x = np.vstack((sol, rvv.reshape(v,1)))
    s = np.matmul(L_inv, x)
    return s

In [22]:
def sign(M,y,rvv,L_inv):   #Generates sign without outsourcing
    signed = False
    while not signed:
        try:
#             rvv = generate_random_vinegar(v)
#             los = sub_vinegar(rvv,F,o,v)
#             M = GF256(los[:, :-1])                         This is omitted here...but M is taken as the first 'o' columns of coefficients values of sub_vinegar functions 
#             c = GF256(los[:, [-1]])                        Similarly, c is the constant generated by vinegar-vinegar substitution. In the NIST documentation as well, yi = vTP(1)v was the constant values. 
#             y = np.subtract(m,c).reshape(o, 1)             
            sol = np.linalg.solve(M,y)
            x = np.vstack((sol, rvv.reshape(v,1)))     
            s = np.matmul(L_inv, x)
            signed = True
        except:
            signed = False
    return s

def verify(P,s,m):
    cmparts= []
    s_T = np.transpose(s)
    for f_p in P:
        cmp1 = np.matmul(s_T,f_p)
        cmp2 = np.matmul(cmp1,s)
        cmparts.append(cmp2[0])
    computed_m = GF256(cmparts)
    return computed_m, np.array_equal(computed_m,m)

In [28]:
import time
print('Without Outsourcing')
o = 20
v = 25

# Consider hash as m in GF(2^8) of size `o`
message, m = generate_gf256_message(o)


# Generate key pair
F, L, L_inv = generate_private_key(o, v)
P = generate_public_key(F, L)

# Sign the message (without outsourcing)

rvv = generate_random_vinegar(v)
los = sub_vinegar(rvv,F,o,v)
M = GF256(los[:, :-1])
c = GF256(los[:, [-1]])
y = np.subtract(m,c)



start_time = time.time()
signature = sign(M,y,rvv,L_inv)
sig_time = time.time() - start_time
print('sign time',sig_time)
# Verify the signature
computed_m, is_valid = verify(P, signature, m)

# Output the results
print("Original message (m):", message)
print("Hash value", m)
print("Generated signature (s):", signature)
print("Computed hash from signature:", computed_m)
print("Signature is valid:", is_valid)

#####################################################################################

Without Outsourcing
sign time 0.007036924362182617
Original message (m): b'\xabbw~\xe3&"L5\xa6\x8f\x04W\x0e\x97\x11~x\x98\xb8'
Hash value [[126]
 [255]
 [173]
 [254]
 [  5]
 [ 93]
 [213]
 [218]
 [141]
 [154]
 [122]
 [129]
 [169]
 [ 31]
 [109]
 [123]
 [145]
 [ 10]
 [181]
 [ 19]]
Generated signature (s): [[ 31]
 [161]
 [191]
 [ 73]
 [189]
 [213]
 [133]
 [249]
 [ 76]
 [  2]
 [243]
 [139]
 [107]
 [ 28]
 [160]
 [ 50]
 [110]
 [ 75]
 [255]
 [192]
 [ 75]
 [172]
 [250]
 [  9]
 [230]
 [239]
 [167]
 [ 85]
 [123]
 [176]
 [141]
 [ 63]
 [242]
 [188]
 [ 77]
 [ 53]
 [175]
 [ 39]
 [222]
 [136]
 [248]
 [ 14]
 [186]
 [149]
 [ 44]]
Computed hash from signature: [[126]
 [255]
 [173]
 [254]
 [  5]
 [ 93]
 [213]
 [218]
 [141]
 [154]
 [122]
 [129]
 [169]
 [ 31]
 [109]
 [123]
 [145]
 [ 10]
 [181]
 [ 19]]
Signature is valid: True


In [29]:
# Sign the message (with outsourcing) for the same message m.
print('With outsourcing')

start_time = time.time()
random_v, Enc_M , Enc_y = sign_outsource(M,y,o)
keygen_time = time.time() - start_time
print('Encryption',keygen_time)

# Send the vectors to the server
np.savetxt('Enc_M.txt', Enc_M, delimiter=',')
np.savetxt('Enc_y.txt', Enc_y, delimiter=',')

With outsourcing
Encryption 0.0020194053649902344


In [30]:
#Result obtained from the server

Enc_sol = np.loadtxt("Enc_sol.txt", delimiter=",")
Enc_sol = Enc_sol.reshape(Enc_sol.shape[0], 1)
Enc_sol = Enc_sol.astype(dtype=np.uint8)
Enc_sol = GF256(Enc_sol)

In [31]:
start_time = time.time()
signature1 = retrace_sign(rvv, Enc_sol,random_v)
sig_time = time.time() - start_time
print('signing time',sig_time)
# Verify the signature
computed_m1, is_valid = verify(P, signature1, m)

# Output the results
print("Original message (m):", message)
print("Hash value", m)
print("Outsourced signature (s):", signature1)
print("Computed hash from signature:", computed_m1)
print("Signature is valid:", is_valid)

signing time 0.003028392791748047
Original message (m): b'\xabbw~\xe3&"L5\xa6\x8f\x04W\x0e\x97\x11~x\x98\xb8'
Hash value [[126]
 [255]
 [173]
 [254]
 [  5]
 [ 93]
 [213]
 [218]
 [141]
 [154]
 [122]
 [129]
 [169]
 [ 31]
 [109]
 [123]
 [145]
 [ 10]
 [181]
 [ 19]]
Outsourced signature (s): [[ 31]
 [161]
 [191]
 [ 73]
 [189]
 [213]
 [133]
 [249]
 [ 76]
 [  2]
 [243]
 [139]
 [107]
 [ 28]
 [160]
 [ 50]
 [110]
 [ 75]
 [255]
 [192]
 [ 75]
 [172]
 [250]
 [  9]
 [230]
 [239]
 [167]
 [ 85]
 [123]
 [176]
 [141]
 [ 63]
 [242]
 [188]
 [ 77]
 [ 53]
 [175]
 [ 39]
 [222]
 [136]
 [248]
 [ 14]
 [186]
 [149]
 [ 44]]
Computed hash from signature: [[126]
 [255]
 [173]
 [254]
 [  5]
 [ 93]
 [213]
 [218]
 [141]
 [154]
 [122]
 [129]
 [169]
 [ 31]
 [109]
 [123]
 [145]
 [ 10]
 [181]
 [ 19]]
Signature is valid: True


In [32]:
np.array_equal(signature,signature1)         #Signature is the true signature, Signature1 is the outsourced signature

True

In [ ]:
Enc_y